In [10]:
# Python calculation for the example and exercise from Lecture 4 notes
import numpy as np
import os
import math
import pandas as pd
import scipy.stats as stat
import scipy.interpolate
import statistics
from statistics import NormalDist

In [11]:
aapl = pd.read_excel("./data/hist_data.xlsm", sheet_name = 'AAPL')
msft = pd.read_excel("./data/hist_data.xlsm", sheet_name = 'MSFT')
f = pd.read_excel("./data/hist_data.xlsm", sheet_name = 'F')
bac = pd.read_excel("./data/hist_data.xlsm", sheet_name = 'BAC')
sofr_curve = pd.read_excel("./data/hist_data.xlsm", sheet_name = 'SofrCurve')

# Extract "T" as a separate array
T_array = sofr_curve["T"].values
df_transposed = sofr_curve[sofr_curve.columns[2:]].T
df_transposed.reset_index(inplace=True)
col_names = ['Date'] + list(sofr_curve['Tenor'])
df_transposed.columns = col_names

sofr_curve = df_transposed[['Date'] + list(df_transposed.columns[7:17])]
sofr_curve.head()
df_stocks = pd.concat([aapl,
                        msft['Adj Close'],
                        f['Adj Close'],
                        bac['Adj Close']],axis=1)
df_stocks.columns = ['Date', 'aapl', 'msft', 'f', 'bac']

df_returns = df_stocks.merge(sofr_curve,
                        on=['Date'],
                        how='outer')\
                        .ffill()\
                        .drop(columns=['Date'])\
                        .pct_change()\
                        .dropna()

df_returns.head()

,aapl,msft,f,bac,1Y,2Y,3Y,4Y,5Y,6Y,7Y,8Y,9Y,10Y
1,-0.017543,-0.017059,0.002244,0.004439,0.011215,0.009846,0.008140,0.007351,0.005731,0.003218,0.000754,-0.001141,-0.002596,-0.003849
2,-0.037305,-0.035368,-0.025373,-0.003039,0.004977,0.010528,0.009553,0.006256,0.003865,0.003057,0.002871,0.002624,0.002407,0.002457
3,-0.042405,-0.026579,0.015314,-0.005542,0.014624,0.024133,0.025373,0.023414,0.020974,0.019620,0.018464,0.016548,0.014541,0.013280
4,-0.001947,0.033326,0.018854,0.025077,-0.008229,-0.010659,-0.010219,-0.007710,-0.005036,-0.002922,-0.000975,0.001187,0.003311,0.005084
5,0.003902,0.029270,0.014064,0.005980,0.009036,0.011631,0.011503,0.009947,0.008235,0.007935,0.008467,0.008905,0.009056,0.008932


## Pricing swap
1. Make a function to calculate payer swap
2. Get discount factors
3. Calculate initial value of swap
4. Calculate PV01:\
    a. change the value of each related zero rate by one bp an take note of PV change

In [12]:
def payer_swap_10y(ls_df, swap_rate):
    """
    Retun value of payer swap
    parameters
        ls_df: list of discount factors
        swap_rate: strike of swap
        sofr: 
    """
    fix_leg = sum(ls_df) * swap_rate * 1    # PVBP * Swap rate * day count fraction
    flt_leg = 1-ls_df[0]

    return flt_leg - fix_leg

swap_rate = .042
ls_df = sofr_curve.iloc[-1][sofr_curve.columns[1:]] # get zero rates for 10 year swap
ls_df = [ np.exp(-df*(i+1)) for i, df in enumerate(ls_df) ] # calculate as discount factors
# initial value of swap
S_0 = payer_swap_10y(ls_df, swap_rate)

### Calculate PV01
Create list of zero rates. For each year, add a bp, save list of resultant DF. Calculate new payer swap PV, and difference from previous day. Save results to list

In [13]:
ls_zero_rates = sofr_curve.iloc[-1][sofr_curve.columns[1:]]
ls_pv01 = []

for i in range(len(ls_zero_rates)):
    ls_df = [r+.0001 if i==j else r for j, r in enumerate(ls_zero_rates)]
    ls_df = [ np.exp(-df*(i+1)) for i, df in enumerate(ls_df) ]

    PV01_partial = payer_swap_10y(ls_df, swap_rate) - S_0
    ls_pv01.append(PV01_partial)

ls_pv01

[9.889092979081804e-05,
 7.631796779650202e-06,
 1.0993001080650266e-05,
 1.4066553566560724e-05,
 1.6854750986838862e-05,
 1.9371056086570082e-05,
 2.1632335943821968e-05,
 2.365322756742172e-05,
 2.544776906476809e-05,
 2.7029751466067786e-05]

## Calculate Mean and Covariance Matrix
1. Use combined df to get mean return of each risk factor
2. Set weight matrix w

In [14]:
mean_ret = df_returns.mean().values
cov_ret = df_returns.cov().values
w = np.concatenate((np.array([1e6]*4),       # join weights for stocks and rates
                   1e8*np.array(ls_pv01)))

mean_1d_ret = mean_ret @ w
var_1d_ret = w @ cov_ret @ w.T

# Parametric VaR
Display possible loss as a positive number

In [15]:
var1d = stat.norm.ppf(.05, loc=mean_1d_ret, scale=np.sqrt(var_1d_ret))

print("="*20, " Return ", "="*20)
print(f"Mean {mean_1d_ret:,.2f}, Variance:  {var_1d_ret:,.2f}, SD:{np.sqrt(var_1d_ret):,.2f}")

print("")
print("")
print("============================================================================================================================")
print(f"Parametric VaR [1d, 95%]: {abs(var1d):,.0f}")
print("============================================================================================================================")

====================  Return  ====================
Mean 550.74, Variance:  2,859,267,252.87, SD:53,472.12


Parametric VaR [1d, 95%]: 87,403


# Monte Carlo Method
Reuse mean and variance from parametric period to create samples
## MC Full revaluation
0. make function to calculate pnl
1. generate samples
2. calculate PnL full revaluation\
    a. calculate initial portfolio values\
    b. calculate stock P_1 values\
    c. calculate swap P_1 values\
    d. calculate P_1 - P_0

In [16]:
# generate samples
num_samples = 1_000_000
samples = np.random.multivariate_normal(mean_ret, 
                                        cov_ret, 
                                        num_samples)
def P_1_calculate(samples):
    """
    Return value of swap and stock
    Parameters:
        samples: in numpy array for all 4 stocks and 10 swap fixing df
    """
    samples_ret = (samples+1) * w.T   # +1 get P_1 for each stock factor
    stock_P_1 = samples_ret[:, :4]
    swap_P_1 = []

    for sample in samples:
        ls_df = np.array(ls_zero_rates).astype(float) * (sample[4:] + 1)  # calculate P_1 zero rates
        ls_df = [ np.exp(-df*(i+1)) for i, df in enumerate(ls_df) ]        # calculate P_1 DF
        # calculate S_1 and append
        swap_P_1.append(payer_swap_10y(ls_df, swap_rate))

    swap_P_1 = 1e8 * np.array(swap_P_1)

    return swap_P_1, stock_P_1

# find initial portfolio value
P_0 = S_0 * 1e8 + 1e6 * 4
# find P_1
swap_P_1, stock_P_1 = P_1_calculate(samples)
P_1 = np.concatenate([stock_P_1, 
                      swap_P_1[:, np.newaxis]],
                      axis=1)\
        .sum(axis=1)

pnl1d_full_sample = P_1 - P_0
var1d_full_mc = np.abs(np.percentile(pnl1d_full_sample, 5))

## MC Sensitiviy Analysis

In [25]:
pnl1d_sen_sample = samples@w.T
var1d_sen_mc = np.abs(np.percentile(pnl1d_sen_sample, 5))

print("")
print("")
print("============================================================================================================================")
print("Monte Carlo VaR:")
print(f"VaR [1d, 95%], Full Revaluation: {var1d_full_mc:,.0f}") 
print("")
print(f"VaR [1d, 95%], Sensitivity: {var1d_sen_mc:,.0f}") 
print("="*20, " Return ", "="*20)
print(f"Mean {pnl1d_sen_sample.mean():,.2f}, Variance:  {pnl1d_sen_sample.var():,.2f}, SD:{pnl1d_sen_sample.std():,.2f}")
print(f"Mean {pnl1d_full_sample.mean():,.2f}, Variance:  {pnl1d_full_sample.var():,.2f}, SD:{pnl1d_full_sample.std():,.2f}")
print("============================================================================================================================")



Monte Carlo VaR:
VaR [1d, 95%], Full Revaluation: 382,029

VaR [1d, 95%], Sensitivity: 87,326
====================  Return  ====================
Mean 585.53, Variance:  2,860,467,990.75, SD:53,483.34
Mean 9,231.76, Variance:  56,384,430,545.95, SD:237,454.06


# Historical Var

In [9]:
hist_samples = df_returns.values
swap_P_1, stock_P_1 = P_1_calculate(hist_samples)
P_1 = np.concatenate([stock_P_1, 
                      swap_P_1[:, np.newaxis]],
                      axis=1)\
        .sum(axis=1)

# Full revaluation for Historical Var
pnl1d_full_hist_sample = P_1 - P_0
var1d_full_hist = np.abs(np.percentile(pnl1d_full_hist_sample, 5))
# Sensitivity impact for Historical Var
pnl1d_sen_hist_sample = hist_samples@w.T
var1d_sen_hist = np.abs(np.percentile(pnl1d_sen_hist_sample, 5))

print("")
print("")
print("============================================================================================================================")
print("Historical VaR:")
print(f"VaR [1d, 99%], Full Revaluation: {var1d_full_hist:,.0f}") 
print("")
print(f"VaR [1d, 99%], Sensitivity: {var1d_sen_hist:,.0f}") 
print("============================================================================================================================")



Historical VaR:
VaR [1d, 99%], Full Revaluation: 383,154

VaR [1d, 99%], Sensitivity: 83,125
